# Workshop Programeren en AI



## Setup
### Imports

In [ ]:
# General imports
import os

# Imports for data handeling
import pandas as pd
from skimpy import clean_columns
import country_converter as coco

# Imports for plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Settings for notebook
pd.set_option('display.max_rows', 40)
sns.set_style()

### Data ophalen

#### Onze dataset bevat 31 kolommen:
- **_year_**: Jaar waarop deze metingen van toepassing zijn.
- **_area_**: Land waar de data vandaan komt.
- **_savanna_fires_**: Emissies van branden in savanne-ecosystemen.
- **_forest_fires_**: Emissies van branden in beboste gebieden.
- **_fires_in_organic_soils_**: Emissies van branden in organische bodems.
- **_fires_in_humid_tropical_forests_**: Emissies van branden in vochtige tropische bossen.
- **_forestland_**: Land bedekt met bossen.
- **_net_forest_conversion_**: Verandering in bosoppervlakte door ontbossing en herbebossing.
- **_crop_residues_**: Emissies door het verbranden of afbreken van gewasresten na de oogst.
- **_rice_cultivation_**: Emissies van methaan die vrijkomen tijdens de rijstteelt.
- **_drained_organic_soils_(co2)_**: Emissies van koolstofdioxide die vrijkomen bij het ontwateren van organische bodems.
- **_agrifood_systems_waste_disposal_**: Emissies van afvalverwerking binnen het agrovoedselsysteem.
- **_ippu_**: Emissies van industriële processen en productgebruik (IPPU).
- **_food_transport_**: Emissies van het transporteren van voedselproducten.
- **_food_processing_**: Emissies van het verwerken van voedselproducten.
- **_food_packaging_**: Emissies van de productie en verwijdering van voedselverpakkingsmaterialen.
- **_food_retail_**: Emissies van de exploitatie van winkels die voedsel verkopen.
- **_food_household_consumption_**: Emissies van voedselconsumptie op huishoudelijk niveau.
- **_pesticides_manufacturing_**: Emissies afkomstig van de productie van pesticiden.
- **_fertilizers_manufacturing_**: Emissies van de productie van meststoffen.
- **_manure_applied_to_soils_**: Emissies van het aanbrengen van dierlijke mest op landbouwgronden.
- **_manure_left_on_pasture_**: Emissies van dierlijke mest op weide- of grasland.
- **_manure_management_**: Emissies van het beheren en behandelen van dierlijke mest.
- **_on_farm_electricity_use_**: Elektriciteitsverbruik op boerderijen.
- **_on_farm_energy_use_**: Energieverbruik op boerderijen.
- **_rural_population_**: Aantal mensen dat in landelijke gebieden woont.
- **_urban_population_**: Aantal mensen dat in stedelijke gebieden woont.
- **_total_population_-_male_**: Totaal aantal mannelijke individuen in de bevolking.
- **_total_population_-_female_**: Totaal aantal vrouwelijke individuen in de bevolking.
- **_total_emission_**: Totale broeikasgasemissies uit verschillende bronnen.
- **_average_temperature_°c_**: De gemiddelde temperatuurstijging (per jaar) in graden Celsius.

#### En we voegen er zelf 4 toe:
- **_continent_**: Continent waar dit gebied zich bevindt.
- **_subregion_**: (Verenigde Naties) subregio waar dit gebied zich bevindt.
- **_total_population_**: Som van de mannelijke en vrouwelijke bevolking.
- **_emission_per_capita_**: Totale emissie gedeeld door de totale bevolking van een gebied.

Alle kolomnamen in de dataset zijn geschreven in [snake_case](https://nl.wikipedia.org/wiki/Snake_case).

**Opdracht 1**: Lees het csv bestand uit naar een tabel (pd.DataFrame), en sla dit tabel op in de `df` variabele. 

In [ ]:
pad_naar_data_file = os.path.join(os.getcwd(), "Agrofood_co2_emission.csv")
assert os.path.exists(pad_naar_data_file), "Het pad naar de data file bestaat niet"

df = pd.read_csv(pad_naar_data_file)  # Read the data from file to a Dataframe
df = clean_columns(df, replace={})

In [ ]:
# Maak je geen zorgen over dit code blok.
# Hier voegen we 4 kolommen toe aan de DataFrame, maar omdat sommige oude landen
# benamingen niet door alles ondersteunt worden moeten we deze eerst corrigeren.
manual_country_fixes = {
    'Belgium-Luxembourg': 'Belgium',
    'Channel Islands': 'United Kingdom',
    'China, Taiwan Province of': 'Taiwan',
    'Netherlands Antilles (former)': 'Netherlands',
    'Pacific Islands Trust Territory': 'Micronesia',
    'Serbia and Montenegro': 'Serbia', 
    'USSR': 'Russia',
    'Yugoslav SFR': 'Serbia',  
}

# Voeg de "continent" en "subregion" kolom toe. Dit doen we om de 236 landen te categoriseren.
cc = coco.CountryConverter()
df['area'] = df['area'].replace(manual_country_fixes)
df["continent"] = cc.convert(names=df["area"], to="continent")
df["subregion"] = cc.convert(names=df["area"], to="UNregion")
assert len(df[df['continent'] == 'not found']['area'].unique()) == 0 
assert len(df[df['subregion'] == 'not found']['area'].unique()) == 0 

# Voeg een "total_population" kolom toe als een som van mannelijke en vrouwelijke populaties.
# Daarna voegen we een kolom toe "emission_per_capita" waar we de totale emissies delen door de totale populatie van een land.
df["total_population"] = df["total_population_male"] + df["total_population_female"]
df["emission_per_capita"] = df["total_emission"] / df["total_population"]

## Bekend worden met de data

Voordat je analyse doet op je data is het belangrijk dat je bekend bent met je data. Zijn er kolommen waar veel waardes missen? Zijn er kolommen die overbodig zijn omdat ze altijd het zelfde zijn? Wat is de correlatie tussen verschillende kolommen?

**Opdracht 2**: print de eerste 5 rijen van de tabel.

In [ ]:
display(df.head(5))

In [ ]:
# Dit code blok maakt een tabel met vier kolommen: "Data Type", "Unieke Waardes", "Null Waardes" en "Non-null Waardes".
#
# - Data Type:          De type van de data in deze kolom (integer, string, etc.)
# - Unieke Waardes:     Hoeveel verschillende waardes er in totaal vorkomen in deze kolom
# - Null Waardes:       Hoevaak deze kolom is leeg gelaten in een rij
# - Non-null Waardes:   Hoevaak deze kolom wel is ingevuld in een rij

dtypes = df.dtypes 
missing_counts = df.isnull().sum()
non_null_counts = df.notnull().sum()
unique_counts = df.nunique()
summary = pd.DataFrame({
    "Data Type": dtypes,
    "Unieke Waardes": unique_counts,
    "# Null Waardes": missing_counts,
    "# Non-null Waardes": non_null_counts,
}).sort_values(by='# Null Waardes', ascending=False)
display(summary)

**Opdracht 3**: Bereken voor elke kolom statestieken over de verdeling van je data. (Denk aan de mean en standaarddeviatie)

_Hint_: Als iets te veel werk lijkt voor iets simpels, is er waarschijnlijk een functie voor in pandas.

In [ ]:
display(df.describe())

In [ ]:
# In dit code blok doen we de correlatie matrix plotten.
# In ons geval zijn correlaties in de tekstvelden niet intressant, maar als je een categorie zou hebben
# (bijvoorbeeld een multiple choice vraag) dan zou je deze eerst moeten encoden naar nummers. Dit kan je ook automatisch laten doen.

figuur = plt.figure(figsize=(19, 15))  # Zet de groote van het figuur

# Maak de correlation matrix, en zet de kolom namen langs de randen
plt.matshow(
    df.drop(labels=["area", "continent", "subregion"], axis=1).corr(),
    fignum=figuur.number,
)
plt.xticks(
    range(df.select_dtypes(["number"]).shape[1]),
    df.select_dtypes(["number"]).columns,
    fontsize=14,
    rotation=90,
)
plt.yticks(
    range(df.select_dtypes(["number"]).shape[1]),
    df.select_dtypes(["number"]).columns,
    fontsize=14,
)

# Zet de legenda naast het plot, en geef het plot een titel
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title("Correlation Matrix", fontsize=18)
plt.show()

## Data analyse

In Python zijn kan je gemakkelijke veel grafieken maken. Hieronder is een voorbeeld te zien met hoeveel de verschillende continenten bijdroegen aan de totale CO2 uistoot over de jaren heen.

In [ ]:
# In did code blok maken we een grafiek dat de bijdrage van verschillende continenten aan de totale CO2
# uitstoot laat zien over de jaren heen.

# Groepeer de rijen van de tabel op basis van "year" en "continent"
groep_jaar_continent = (
    df.groupby(["year", "continent"])["total_emission"].sum().unstack(fill_value=0)
)

# Om de stapel volgorde in de grafiek wat mooier te maken sorteren we op de 
# waardes van het laatste jaar. Zo zal de grootste uitstooter boven staan.
sorted_subregions = groep_jaar_continent.iloc[-1].sort_values().index.tolist()
grouped_year_sorted = groep_jaar_continent[sorted_subregions]

# Plot de grafiek.
fig, ax = plt.subplots(figsize=(20, 10))
ax.stackplot(
    grouped_year_sorted.index,
    grouped_year_sorted.T.values,
    labels=grouped_year_sorted.columns,
)

# Voeg een titel, assen titels en een legenda toe aan de grafiek.
plt.title("Totale emissies over de jaren per continent")
plt.xlabel("Jaar")
plt.ylabel("Totale Emissies")
plt.legend(title="Subregion", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True)
plt.show()

Van hier verder gaan we alleen nog met data van Europa werken.

**Opdracht 4**: Maak een tabel aan met alleen de gegevens van Europa er in en sla deze op in de `df_europe` variabele. Als je dit hebt gedaan, doe dan ook even checken of het gelukt is ;)

In [ ]:
df_europe = df[df["continent"] == "Europe"]
assert df_europe["continent"].nunique() == 1

In [ ]:
pp = sns.jointplot(data=df_europe, x="food_household_consumption", y="agrifood_systems_waste_disposal", hue="subregion")

Nu Jullie wat voorbeelden hebben gezien van wat je allemaal kan mogen jullie zelf aan de slag.

**Opdracht 5**: bezoek de [Matplotlib site](https://matplotlib.org/stable/plot_types/index.html) en maak zelf een plot van de data, zie je nog ergens interresante relaties.